In [21]:
import pandas as pd
import numpy as np


In [77]:
# get unique values of columns needed
albums_col = ["_id", "id_artist"]
artists_col = ["_id", "type", "gender", "members"]
songs_col = ["id_album", "genre"]

# assuming the data folders are in your local folder
albums_data = pd.read_csv("../data/wasabi_albums.csv", usecols=albums_col)
artists_data = pd.read_csv("../data/wasabi_artists.csv", usecols=artists_col)
songs_data = pd.read_csv("../data/wasabi_songs.csv",
                         sep="\t", usecols=songs_col)


In [75]:
display("albums", albums_data.head(2))
display("artists", artists_data.head(2))
display("songs", songs_data.head(2))


'albums'

,_id,id_artist
0,ObjectId(5714debb25ac0d8aee34d59a),ObjectId(56d7e91b6b60c09814f93e4a)
1,ObjectId(5714debb25ac0d8aee34d59b),ObjectId(56d7e91b6b60c09814f93e4a)


'artists'

,_id,gender,members,type
0,ObjectId(56d7e91b6b60c09814f93e4a),NaN,"[{""id_member_musicbrainz"":""3ec05e94-bf6e-439f-...",Group
1,ObjectId(56d7e91c6b60c09814f93e4c),NaN,"[{""id_member_musicbrainz"":""82bd3da4-7085-40b8-...",Group


'songs'

,genre,id_album
0,NaN,ObjectId(5714debb25ac0d8aee34d59a)
1,NaN,ObjectId(5714debb25ac0d8aee34d59a)


In [329]:
# df to export in the end
sankey_col = ["source", "target", "value"]
sankey_df = pd.DataFrame(columns=sankey_col)


In [330]:
artists_data\
    .groupby(["type", "gender"], dropna=False)\
    .count()


_id  members
type      gender                
Character Female      4        4
          Male       15       15
          NaN         6        6
Choir     NaN        44       44
Group     NaN     29806    29806
Orchestra NaN        30       30
Other     Female      3        3
          Male        1        1
          NaN        17       17
Person    Female   5867     5867
          Male    14281    14281
          Other       8        8
          NaN      4108     4108
NaN       Female      7        7
          Male       12       12
          NaN     23283    23283

# Sankey type to gender

In [331]:
df = artists_data.copy()\
    .groupby(["type", "gender"], dropna=False).count()
display(df)
df = df.reset_index()
display(df)
df = df.drop(columns=["location", "members"], errors="ignore")
df = df.rename(columns={"_id": "value"})
df = df.fillna({col: "unknown_"+col for col in df.columns})
df = df.replace({col: {"Other": "unknown_"+col for _ in df.columns}
                for col in df.columns})
df


_id  members
type      gender                
Character Female      4        4
          Male       15       15
          NaN         6        6
Choir     NaN        44       44
Group     NaN     29806    29806
Orchestra NaN        30       30
Other     Female      3        3
          Male        1        1
          NaN        17       17
Person    Female   5867     5867
          Male    14281    14281
          Other       8        8
          NaN      4108     4108
NaN       Female      7        7
          Male       12       12
          NaN     23283    23283

,type,gender,_id,members
0,Character,Female,4,4
1,Character,Male,15,15
2,Character,NaN,6,6
3,Choir,NaN,44,44
4,Group,NaN,29806,29806
5,Orchestra,NaN,30,30
6,Other,Female,3,3
7,Other,Male,1,1
8,Other,NaN,17,17
9,Person,Female,5867,5867


,type,gender,value
0,Character,Female,4
1,Character,Male,15
2,Character,unknown_gender,6
3,Choir,unknown_gender,44
4,Group,unknown_gender,29806
5,Orchestra,unknown_gender,30
6,unknown_type,Female,3
7,unknown_type,Male,1
8,unknown_type,unknown_gender,17
9,Person,Female,5867


In [332]:
df.columns = sankey_col
sankey_df = sankey_df.append(df)


# Sankey gender to nb_albums

In [333]:
display(albums_data.head(2), artists_data.head(2))


,_id,id_artist
0,ObjectId(5714debb25ac0d8aee34d59a),ObjectId(56d7e91b6b60c09814f93e4a)
1,ObjectId(5714debb25ac0d8aee34d59b),ObjectId(56d7e91b6b60c09814f93e4a)


,_id,gender,members,type
0,ObjectId(56d7e91b6b60c09814f93e4a),NaN,"[{""id_member_musicbrainz"":""3ec05e94-bf6e-439f-...",Group
1,ObjectId(56d7e91c6b60c09814f93e4c),NaN,"[{""id_member_musicbrainz"":""82bd3da4-7085-40b8-...",Group


In [334]:
df_albums = albums_data\
    .copy()\
    .groupby("id_artist").count()\
    .rename(columns={"_id": "nb_albums"})\
    .reset_index()
df_albums["id_artist"] = [element if element.startswith("ObjectId(") else "ObjectId("+element+")" for element in df_albums["id_artist"]]

df_albums

,id_artist,nb_albums
0,ObjectId(56d7e9296b60c09814f93e71),1
1,ObjectId(56d7e91b6b60c09814f93e4a),6
2,ObjectId(56d7e91c6b60c09814f93e4b),1
3,ObjectId(56d7e91c6b60c09814f93e4c),7
4,ObjectId(56d7e91d6b60c09814f93e4d),2
...,...,...
77487,ObjectId(56d99879cc2ddd0c0f6bf405),8
77488,ObjectId(56d9987dcc2ddd0c0f6bf406),13
77489,ObjectId(56d9987ecc2ddd0c0f6bf407),21
77490,ObjectId(57c92566e5c453a411c771f4),10


In [335]:
df_artists = artists_data\
    .copy()\
    .rename(columns={"_id":"id_artist"})\
    [["id_artist", "gender"]]\

df_artists

,id_artist,gender
0,ObjectId(56d7e91b6b60c09814f93e4a),NaN
1,ObjectId(56d7e91c6b60c09814f93e4c),NaN
2,ObjectId(56d7e91d6b60c09814f93e4e),NaN
3,ObjectId(56d7e91e6b60c09814f93e50),NaN
4,ObjectId(56d7e91e6b60c09814f93e52),NaN
...,...,...
77487,ObjectId(56d997b1cc2ddd0c0f6bf2d2),NaN
77488,ObjectId(56d997b3cc2ddd0c0f6bf2d3),NaN
77489,ObjectId(56d997b3cc2ddd0c0f6bf2d4),NaN
77490,ObjectId(56d997b3cc2ddd0c0f6bf2d5),NaN


In [336]:
l1 = np.array(df_albums.index)
l2 = np.array(df_artists.index)
set(l1).symmetric_difference(set(l2))

set()

In [337]:
df = df_artists.copy()\
    .merge(df_albums, on="id_artist", how="outer")
df = df.fillna({col: "unknown_"+col for col in df.columns})\
    .drop(columns="id_artist")\
    .value_counts()\
    .reset_index()
df.columns = sankey_col
df

,source,target,value
0,unknown_gender,1,37800
1,Male,1,8090
2,unknown_gender,2,5915
3,unknown_gender,3,3763
4,Female,1,2990
...,...,...,...
180,Female,55,1
181,Female,53,1
182,Female,52,1
183,Female,51,1


In [338]:
sankey_df = sankey_df.append(df)
sankey_df

,source,target,value
0,Character,Female,4
1,Character,Male,15
2,Character,unknown_gender,6
3,Choir,unknown_gender,44
4,Group,unknown_gender,29806
...,...,...,...
180,Female,55,1
181,Female,53,1
182,Female,52,1
183,Female,51,1


In [339]:
sankey_df.to_csv("sankey.csv", index=False)